# **模型訓練（分類問題）**
此份程式碼會講解針對分類型任務在模型訓練上需要注意的細節。

## 本章節內容大綱
* ### [創建資料集／載入資料集（Dataset Creating/ Loading）](#DatasetCreating/Loading)
* ### [資料前處理（Data Preprocessing）](#DataPreprocessing)
* ### [模型建置（Model Building）](#ModelBuilding)
* ### [模型訓練（Model Training）](#ModelTraining)
* ### [模型評估（Model Evaluation）](#ModelEvaluation)
---

## 匯入套件

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

<a name="DatasetCreating/Loading"></a>
## 創建資料集／載入資料集（Dataset Creating / Loading）

In [ ]:
train_df = pd.read_csv('./Data/FilmComment_train.csv')
test_df = pd.read_csv('./Data/FilmComment_test.csv')

In [ ]:
train_df.head()

* #### 電影評論資料集
訓練集，測試集分別為 6250，2500 筆，9997 種常用字詞，若在同一則評論中出現該字詞為 1，若否則為 0，y_label 標記評價正面與否。

In [ ]:
X_df = train_df.iloc[:, :-1].values
y_df = train_df.y_label.values
X_test = test_df.iloc[:, :-1].values
y_test = test_df.y_label.values

<a name="DataPreprocessing"></a>
## 資料前處理（Data Preprocessing）

* ### 資料正規化（Data Normalization）
由於此資料集的數值範圍都介於 0-1，並且皆是以相同意義轉換特徵值，因此也可以使用原始的數值作為訓練資料。

* ### 資料切分（Data Splitting）

In [ ]:
# train, valid/test dataset split
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = \
    train_test_split(X_df, y_df, test_size=0.1, random_state=17)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_valid shape: {y_valid.shape}')

<a name="ModelBuilding"></a>
## 模型建置（Model Building）

In [ ]:
keras.backend.clear_session()  # 重置 keras 的所有狀態
tf.random.set_seed(17)  # 設定 tensorflow 隨機種子

model = keras.models.Sequential()
model.add(layers.Dense(16,  # 神經元個數
                       input_shape=X_train[0].shape,  # 輸入形狀
                       activation='relu'))  # 激活函數
model.add(layers.Dense(16,
                       activation='relu'))
model.add(layers.Dense(1,
                       activation='sigmoid'))

model.summary()

![](https://i.imgur.com/75GQ9gz.png)

## 模型訓練（Model training）

* ### 模型編譯（model compile）
設定模型訓練時，所需的優化器 (optimizer)、損失函數 (loss function)、評估指標 (metrics)

In [ ]:
model.compile(optimizer='rmsprop',  # default: RMSprop(learning_rate=0.001)
              loss='binary_crossentropy',  # 針對二元分類問題的損失函數
              metrics='acc')  # 評估指標: 準確率

![](https://i.imgur.com/GazyxvF.png)

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=20,
                    validation_data=(X_test, y_test))

<a name="ModelEvaluation"></a>
## 模型評估（Model Evaluation）

* ### 視覺化訓練過程的評估指標 （Visualization）

In [ ]:
# type(history.history) = dictionary
print(history.history.keys())

In [ ]:
train_loss = history.history['loss']
train_acc = history.history['acc']

valid_loss = history.history['val_loss']
valid_acc = history.history['val_acc']

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_loss)), train_loss, label='train_loss')
plt.plot(range(len(valid_loss)), valid_loss, label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(train_acc)), train_acc, label='train_acc')
plt.plot(range(len(valid_acc)), valid_acc, label='valid_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

* ### 模型預測（Model predictions）

In [ ]:
# predict all test data
pred = model(X_test)
print(pred)

In [ ]:
# use threshold to obtain binary class
pred_class = model(X_test) > 0.5
print(tf.cast(pred_class, tf.int32))

* ### 視覺化結果

In [ ]:
plt.figure(figsize=(15, 4))
plt.scatter(range(pred.shape[0]), pred)
plt.hlines(0.5, 0, pred.shape[0], colors='red', label='y=0.5')
plt.legend()

----------------
此範例是二元分類，y 的表示方式可用一維陣列，分別以 0, 1 表示兩個類別（正面，負面評價）
![](https://i.imgur.com/QYp5PVQ.png)

**若是多元分類又該如何表示？**以多個維度的 One-Hot Encoding 方式表示多元分類標籤
![](https://i.imgur.com/vJcdRBj.png)

**對訓練有何影響？**
跟 y 最直接相關的就是 Loss function，間接影響到模型輸出的維度
![](https://i.imgur.com/gbEMjDl.png)

----------------------------

## 多元分類（Multi-class classification）

### 創建資料集／載入資料集（Dataset Creating / Loading）

In [ ]:
train_df = pd.read_csv('./Data/FilmComment_train.csv')
test_df = pd.read_csv('./Data/FilmComment_test.csv')

In [ ]:
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1].values
y_test = test_df.iloc[:, -1].values

In [ ]:
# Convert to One-Hot encoding
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

### 模型建置（Model Building）

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(17)

model = keras.models.Sequential()
model.add(.layers.Dense(16,
                        input_shape=X_train[0].shape,
                        activation='relu'))
model.add(layers.Dense(16,
                       activation='relu'))
model.add(layers.Dense(2,
                       activation='softmax'))

model.summary()

### 模型訓練（Model training）

* #### 模型編譯（model compile）
設定模型訓練時，所需的優化器 (optimizer)、損失函數 (loss function)

In [ ]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics='acc')

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=20,
                    validation_data=(X_test, y_test))

### 模型評估（Model evalutation）

* #### 視覺化訓練過程的評估指標 （Visualization）

In [ ]:
# type(history.history) = dictionary
print(history.history.keys())

In [ ]:
train_loss = history.history['loss']
train_acc = history.history['acc']

valid_loss = history.history['val_loss']
valid_acc = history.history['val_acc']

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_loss)), train_loss, label='train_loss')
plt.plot(range(len(valid_loss)), valid_loss, label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(train_acc)), train_acc, label='train_acc')
plt.plot(range(len(valid_acc)), valid_acc, label='valid_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

* ### 模型預測（Model predictions）

In [ ]:
# predict all test data
pred = model(X_test)
print(pred)

In [ ]:
pred = tf.argmax(model(X_test), axis=-1)  # choose maximum probability of index
print(pred)

---
### Remark
**Classification task**
![](https://i.imgur.com/vva88Jj.png)

### Quiz
請試著利用 Data/pkgo_train.csv 做多元分類問題，預測五個種類的 pokemon，並調整模型（網路層數、神經元數目）得到更高的準確度。

pkgo_train 為 Pokemon go 中 pokemon 出沒狀態描述的資料集，欄位說明如下：
* latitude, longitude: 位置（經緯度）
* local.xx: 時間（擷取格式 mm-dd'T'hh-mm-ss.ms'Z'）
* appearedTimeOfDay: night, evening, afternoon, morning 四種時段
* appearedHour/Minute: 當地小時／分鐘
* appearedDayOfWeek: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday
* appearedDay/Month: 當地日期／月份
* terrainType: 地形種類
* closeToWater: 是否接近水源（100 公尺內）
* city: 城市
* continent: 洲別
* weather: 天氣種類（Foggy Clear, PartlyCloudy, MostlyCloudy, Overcast, Rain, BreezyandOvercast, LightRain, Drizzle, BreezyandPartlyCloudy, HeavyRain, BreezyandMostlyCloudy, Breezy, Windy, WindyandFoggy, Humid, Dry, WindyandPartlyCloudy, DryandMostlyCloudy, DryandPartlyCloudy, DrizzleandBreezy, LightRainandBreezy, HumidandPartlyCloudy, HumidandOvercast, RainandWindy）
* temperature: 攝氏溫度
* windSpeed: 風速（km/h）
* windBearing: 風向
* pressure: 氣壓
* sunrise/sunsetXX: 日出日落相關訊息
* population_density: 人口密集度
* urban/suburban/midurban/rural: 出沒過的地點城市程度（人口密集度小於 200 為 rural, 大於等於 200 且小於 400 為 midUrban, 大於等於400 且小於 800 為 subUrban, 大於 800 為 urban）
* gymDistanceKm: 最近道館的距離
* gymInxx: 道館是否在指定距離內
* cooc1-cooc151: 是否有其他 pokemon 在 24 小時內，出現在周圍 100 公尺之內
* category: 種類